In [1]:
#Importamos las librerias necesarias
import pandas as pd
import time 
import numpy as np
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### <span style="color:green">MISTER MASCOTAS</span>

In [2]:
#Creamos un DF vacio
aux_1=pd.DataFrame()

In [3]:
#Convertimos el DF a un archivo de excel ".xlsx"
aux_1.to_excel("df_mister.xlsx",index=False)

#Verificamos que se haya creado el archivo en la misma carpeta en donde está este código

In [4]:
#Función que recibe como parámetro la variable "alimento"
#En el caso de esta tienda tuvimos que definir 2 funciones ya que para obtener el producto "alimento"
#la url cambiaba, además de que los productos se dividian en 2 urls distintas
def Buscador_Precios_Mister_Alimento(producto):
    
    #1.INGRESAMOS A LA PÁGINA WEB
    #Primera página url:
    path ="/Users/Carmen Rosa/Documents/teoria-LP2/web/chromedriver"
    driver=webdriver.Chrome(path)
    url= "https://www.mistermascotas.com.mx/collections/"+producto+"-para-perro"
    driver.get(url)
    
    #2.ACCEDEMOS A LOS ELEMENTOS QUE CONTIENEN LOS DATOS QUE QUEREMOS DE LA PÁGINA WEB
    productos= driver.find_elements_by_class_name("product-grid-item")
    
    #3.ACCEDEMOS A LAS URL'S ALMACENADAS EN LA VARIABLE PRODUCTOS
    #Para obtener varios url´, se utilizará un ciclo for
    #Declaramos una lista vacía
    lista_urls=list()
    #Para cada una de los productos hasta la longitud de la lista:
    for i in range(len(productos)):
        try:
            #En cada una de las entradas se va a buscar la etiqueta "a" y el atributo "href"
            lista_urls.append(productos[i].find_element_by_tag_name("a").get_attribute("href"))
        except:
            #En caso de error se va a agregar a la lista un registro NaN
            lista_urls.append(np.nan)
    
    #4.ACCEDEMOS A LOS NOMBRES ALMACENADAS EN LA VARIABLE PRODUCTOS
    #Ahora hacemos lo mismo pero para obtener los NOMBRES
    lista_nombres=list()
    for i in range(len(productos)):
        try:
            lista_nombres.append(productos[i].find_elements_by_tag_name("a")[1].text)  
        except:
            lista_nombres.append(np.nan)
            
    #5.ACCEDEMOS A LOS PRECIOS BASE Y PROMO EN LA VARIABLE PRODUCTOS
    #En el caso de este autoservicio hay una manera de obtener el precio con y sin descuento en la misma consulta 
    #de una clase, por lo cual sólo obtenemos una lista de precios
    #Fer me permitió hacer esto ya que había problemitas cuando obteniamos por separado las listas con y sin descuento
    lista_precios=list()

    for i in range(len(productos)):

        try:
            lista_precios.append(productos[i].find_elements_by_class_name("money")[0].text)
        #Manadará NaN cuando el producto esté agotado
        except:
            lista_precios.append(np.nan)
    
    #6.CREAR DATAFRAME A PARTIR DE LOS DATOS OBTENIDOS
    #Creamos el DataFrame
    df_mister =pd.DataFrame({"Nombre_producto":lista_nombres,
                             "Producto":producto,
                             "Precios_de_lista":lista_precios,
                             "url":lista_urls})

    #A través de la librería "time" se pone la fecha asociada al día en el que se hace la extracción de la información
    df_mister["Fecha_de_consulta"]= time.strftime("%d/%m/%y")
    
    #Colocamos el nombre del autoservicio de manera manual en el DF
    df_mister["Autoservicio"]="Mister Mascotas"
    
    #Para reacomodar las columnas del DF según el las especificaciones del proyecto final
    df_mister = df_mister[["Fecha_de_consulta","Autoservicio","Producto","Nombre_producto","Precios_de_lista","url"]]
    
    #7.ESCRIBIR SOBRE EXCEL
    #Convertimos el excel en un DF
    datos_webscraper=pd.read_excel("df_mister.xlsx")

    #Concatenamos el DF "vacío" con el DF de Mister Mascotas con el que estuvimos trabajando
    datos_webscraper= pd.concat([datos_webscraper,df_mister],axis=0)

    #Finalmente convertimos de nuevo el DF en un archivo excel con el mismo nombre
    datos_webscraper.to_excel("df_mister.xlsx",index=False)
#———————————————————————————————————————————————————————————————————————————————————————————————————————————————————   
    #1.INGRESAMOS A LA PÁGINA WEB
    #segunda página url
    path ="/Users/Carmen Rosa/Documents/teoria-LP2/web/chromedriver"
    driver=webdriver.Chrome(path)
    url= "https://www.mistermascotas.com.mx/collections/"+producto+"-para-perro?page=2"
    driver.get(url)
    
    #2.ACCEDEMOS A LOS ELEMENTOS QUE CONTIENEN LOS DATOS QUE QUEREMOS DE LA PÁGINA WEB
    productos= driver.find_elements_by_class_name("product-grid-item")
    
    #3.ACCEDEMOS A LAS URL's ALMACENADAS EN LA VARIABLE PRODUCTOS
    lista_urls=list()
    for i in range(len(productos)):
        try:
            lista_urls.append(productos[i].find_element_by_tag_name("a").get_attribute("href"))
        except:
            lista_urls.append(np.nan)
    
    #4.ACCEDEMOS A LOS NOMBRES ALMACENADAS EN LA VARIABLE PRODUCTOS
    lista_nombres=list()
    for i in range(len(productos)):
        try:
            lista_nombres.append(productos[i].find_elements_by_tag_name("a")[1].text)  #los nombres están en el 3er lugar
        except:
            lista_nombres.append(np.nan)
            
    #5.ACCEDEMOS A LOS PRECIOS BASE Y PROMO EN LA VARIABLE PRODUCTOS
    lista_precios=list()

    for i in range(len(productos)):

        try:
            lista_precios.append(productos[i].find_elements_by_class_name("money")[0].text)

        except:
            lista_precios.append(np.nan)
    
    #6.CREAR DATAFRAME A PARTIR DE LOS DATOS OBTENIDOS
    df_mister =pd.DataFrame({"Nombre_producto":lista_nombres,
                             "Producto":producto,
                             "Precios_de_lista":lista_precios,
                             "url":lista_urls})

    df_mister["Fecha_de_consulta"]= time.strftime("%d/%m/%y")

    df_mister["Autoservicio"]="Mister Mascotas"
    
    df_mister = df_mister[["Fecha_de_consulta","Autoservicio","Producto","Nombre_producto","Precios_de_lista","url"]]
    
    #7.ESCRIBIR SOBRE EXCEL
    datos_webscraper=pd.read_excel("df_mister.xlsx")

    datos_webscraper= pd.concat([datos_webscraper,df_mister],axis=0)

    datos_webscraper.to_excel("df_mister.xlsx",index=False)
    
    
    #8.SALIR DEL DRIVER
    #Para salir del driver (cerrar ventana de chrome con la que estámos trabajando)
    driver.quit()
    #Al final se regresa el DF de Mister Mascotas
    return df_mister

#Para cada producto que esté en la lista se va a llamar a la función
#Para esta función sólo extraerá cuando se coloque "alimento"
for productos in ["alimento"]:
    Buscador_Precios_Mister_Alimento(productos)
    


C:\Users\CARMEN~1\AppData\Local\Temp/ipykernel_29056/3707045566.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(path)
C:\Users\CARMEN~1\AppData\Local\Temp/ipykernel_29056/3707045566.py:14: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  productos= driver.find_elements_by_class_name("product-grid-item")
C:\Users\Carmen Rosa\.conda\lib\site-packages\selenium\webdriver\remote\webelement.py:341: UserWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead")
C:\Users\Carmen Rosa\.conda\lib\site-packages\selenium\webdriver\remote\webelement.py:359: UserWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  warnings.war

In [5]:
#Función que recibe como parámetro el producto de interés
#Totalmente igual a la funcion anterior
def Buscador_Precios_Mister_Mascotas(producto):
    
    #1.INGRESAMOS A LA PÁGINA WEB
    path ="/Users/Carmen Rosa/Documents/teoria-LP2/web/chromedriver"
    driver=webdriver.Chrome(path)
    url= "https://www.mistermascotas.com.mx/collections/"+producto+"-para-perros"
    driver.get(url)
    
    #2.ACCEDEMOS A LOS ELEMENTOS QUE CONTIENEN LOS DATOS QUE QUEREMOS DE LA PÁGINA WEB
    productos= driver.find_elements_by_class_name("product-grid-item")
    
    #3.ACCEDEMOS A LAS URL'S ALMACENADAS EN LA VARIABLE PRODUCTOS
    lista_urls=list()
    for i in range(len(productos)):
        try:
            lista_urls.append(productos[i].find_element_by_tag_name("a").get_attribute("href"))
        except:
            lista_urls.append(np.nan)
    
    #4.ACCEDEMOS A LOS NOMBRES ALMACENADAS EN LA VARIABLE PRODUCTOS
    lista_nombres=list()
    for i in range(len(productos)):
        try:
            lista_nombres.append(productos[i].find_elements_by_tag_name("a")[1].text) #los nombres están en el 3er lugar
        except:
            lista_nombres.append(np.nan)
            
    #5.ACCEDEMOS A LOS PRECIOS BASE Y PROMO EN LA VARIABLE PRODUCTOS
    lista_precios=list()

    for i in range(len(productos)):

        try:
            lista_precios.append(productos[i].find_elements_by_class_name("money")[0].text)

        except:
            lista_precios.append(np.nan)
    
    #6.CREAR DATAFRAME A PARTIR DE LOS DATOS OBTENIDOS
    df_mister =pd.DataFrame({"Nombre_producto":lista_nombres,
                             "Producto":producto,
                             "Precios_de_lista":lista_precios,
                             "url":lista_urls})

    df_mister["Fecha_de_consulta"]= time.strftime("%d/%m/%y")

    df_mister["Autoservicio"]="Mister Mascotas"
    
    df_mister = df_mister[["Fecha_de_consulta","Autoservicio","Producto","Nombre_producto","Precios_de_lista","url"]]
    
    #7.ESCRIBIR SOBRE EXCEL
    datos_webscraper=pd.read_excel("df_mister.xlsx")

    datos_webscraper= pd.concat([datos_webscraper,df_mister],axis=0)

    datos_webscraper.to_excel("df_mister.xlsx",index=False)
    
    #8.SALIR DEL DRIVER
    driver.quit()
    return df_mister

#Para esta función sí se pueden obtener otros productos para perros, además de "camas" y "juguetes"
for productos in ["camas","juguetes"]:
    Buscador_Precios_Mister_Mascotas(productos)
    
#Se va lee la hoja de cálculo de excel
df_mister=pd.read_excel("df_mister.xlsx")
df_mister  

C:\Users\CARMEN~1\AppData\Local\Temp/ipykernel_29056/3066621077.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(path)
C:\Users\CARMEN~1\AppData\Local\Temp/ipykernel_29056/3066621077.py:12: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  productos= driver.find_elements_by_class_name("product-grid-item")


,Fecha_de_consulta,Autoservicio,Producto,Nombre_producto,Precios_de_lista,url
0,19/03/22,Mister Mascotas,alimento,Royal Canin Urinary SO - Alimento para Perro,"$ 1,949",https://www.mistermascotas.com.mx/collections/...
1,19/03/22,Mister Mascotas,alimento,Royal Canin Gastro Intestinal Low Fat - Alimen...,"$ 1,997",https://www.mistermascotas.com.mx/collections/...
2,19/03/22,Mister Mascotas,alimento,ProPlan HA Hydrolyzed - Alimento para Perro Ve...,NaN,https://www.mistermascotas.com.mx/collections/...
3,19/03/22,Mister Mascotas,alimento,Royal Canin Renal Support S - Alimento para Perro,"$ 1,429",https://www.mistermascotas.com.mx/collections/...
4,19/03/22,Mister Mascotas,alimento,ProPlan NF Kidney Function - Alimento para Per...,"$ 1,429",https://www.mistermascotas.com.mx/collections/...
...,...,...,...,...,...,...
150,19/03/22,Mister Mascotas,juguetes,Kong Juguete para Morder para Perros Dotz Circle,$ 199,https://www.mistermascotas.com.mx/collections/...
151,19/03/22,Mister Mascotas,juguetes,Kong Juguete para Perro Palo para Morder y Lanzar,$ 249,https://www.mistermascotas.com.mx/collections/...
152,19/03/22,Mister Mascotas,juguetes,Peluche para Perro Kong Dragón Wild Knots,$ 349,https://www.mistermascotas.com.mx/collections/...
153,19/03/22,Mister Mascotas,juguetes,Hueso de Juguete para Perro Kong,$ 179,https://www.mistermascotas.com.mx/collections/...


In [ ]:
#PRECIOS (CADENAS) A FLOATS

#Recibe todos los registros del DF 
def precios_floats_1(datos):

    #Eliminamos el signo en los precios
    
    #1ª columna
    for i in range(len(datos["Precios_de_lista"])):
        try:
            datos["Precios_de_lista"].iloc[i]=datos["Precios_de_lista"].iloc[i].strip("$")
        except:
            pass
          
    #Quitamos la separación de comas para miles
    datos["Precios_de_lista"]=datos["Precios_de_lista"].replace(",","",regex=True)
   
    #Convertimos los precios a numéricos    
    datos['Precios_de_lista'] = pd.to_numeric(datos['Precios_de_lista'], errors='coerce')

    datos.to_excel("df_mister_limpio.xlsx",index=False)
        
    #Visualizamos los tipos de datos
    print(datos.dtypes)
    return datos

#Verificamos que el DF ahora cuente con precios de tipo float
precios_floats_1(df_mister)

df_mister=pd.read_excel("df_mister_limpio.xlsx")
df_mister

### <span style="color:orange">WALMART</span>

In [ ]:
#Nos apoyamos de una variable aux para crear la hoja de cálculo 
aux_2 = pd.DataFrame()
aux_2.to_excel("df_walmart.xlsx",index=False)

In [ ]:
def Buscador_Precios_Selenium_Walmart(producto):  
    #Primero definimos a path con la ruta de acceso de chromedriver en nuestra maquina  
    path = "/Users/Carmen Rosa/Documents/teoria-LP2/web/chromedriver"
    driver = webdriver.Chrome(path)
    
    #Ingresamos a la página que queremos y copiamos el link de ella
    URL = "https://www.walmart.com.mx/productos?Ntt="+producto
    #Así podemos usar nuestro driver en ella
    driver.get(URL)
    
    #Accedemos a los elementos que contienen los datos de la página que queremos con ayuda del webdriver

    productos= driver.find_elements_by_class_name("grid_product__30OQa")
    
    #Ahora accedemos a los URLS almacenados en productos, para ello creamos una lista vacía
    #Iteramos en el rango de la página, en el caso de Walmart la pagína muestra 48 productos para así agregar los URL's de
    # los productos a la lista lurls
    lista_urls=list()
    for i in range(len(productos)):
        try:
            lista_urls.append(productos[i].find_element_by_tag_name("a").get_attribute("href"))
        except:
            lista_urls.append(np.nan)
    #Entonces vamos a acceder a los nombres de los productos almacenados en productos, creamos otra lista vacía
    #Iteramos de nuevo en el rango de la página y agregamos los nombres de los productos en la lista lnombres
    lista_nombres=list()
    for i in range(len(productos)):
        try:
            lista_nombres.append(productos[i].find_elements_by_tag_name("a")[1].text)
        except:
            lista_nombres.append(np.nan)
    #Por últio vamos a acceder a los precios de los productos almacenados en productos, hacemos 
    #dos listas, ya que hay precios con promoción
    #Iteramos por último en el rango de la página y agregamos los precios en la lista lprecios, mientras que los 
    #precios que disminuyen por alguna promoción se agregarán a lpromos
    lista_precios=list()
    lista_promos=list()
    for i in range(len(productos)):
        try:
            lista_precios.append(productos[i].find_elements_by_class_name("product_price__2NBjj")[0].text.split("\n")[0])
        except:
            lista_precios.append(np.nan)
        try:
            lista_promos.append(productos[i].find_elements_by_class_name("product_price__2NBjj")[0].text.split("\n")[1])
        except:
            lista_promos.append(np.nan)
    #Programamos el encabezado de la hoja de excel y el llenado de está         
    df_walmart =pd.DataFrame({"Nombre_producto":lista_nombres,
                              "Producto":producto,
                              "url":lista_urls,
                              "Precios_de_lista":lista_precios,
                              "Precios_en_promoción":lista_promos})
   
    df_walmart["Fecha_de_consulta"]= time.strftime("%d/%m/%y")
    
    df_walmart["Autoservicio"]="Walmart"

    df_walmart = df_walmart[["Fecha_de_consulta","Autoservicio","Producto","Nombre_producto","Precios_de_lista","Precios_en_promoción","url"]]
    
    df_walmart=df_walmart.reset_index(drop=True)

    datos_webscraper=pd.read_excel("df_walmart.xlsx")

    datos_webscraper= pd.concat([datos_webscraper,df_walmart],axis=0)

    datos_webscraper.to_excel("df_walmart.xlsx",index=False)

    driver.quit()
    return df_walmart

#Colocamos los productos que queremos "exportar"
for productos in ["croquetas para perro","camas para perro","juguetes para perro"]:
    Buscador_Precios_Selenium_Walmart(productos)

#Se lee la hoja de cálculo de excel
df_walmart=pd.read_excel("df_walmart.xlsx")
df_walmart

In [ ]:
#PRECIOS (CADENAS) A FLOATS
def precios_floats_2(datos): 
    #Hacemos un iterador en la primera columna para quitar el signo de pesos en la columna "Precios_de_lista"
    for i in range(len(datos["Precios_de_lista"])):
        try:
            datos["Precios_de_lista"].iloc[i]=datos["Precios_de_lista"].iloc[i].strip("$")
        except:
            pass
    
    #Y hacemos un iterador pero en la segunda columna que es la de "Precio_en_promoción"
    for i in range(len(datos["Precios_en_promoción"])):
        try:
            datos["Precios_en_promoción"].iloc[i]=datos["Precios_en_promoción"].iloc[i].strip("$")
        except:
            pass
        
    #Quitamos la separacion de comas para miles de la siguiente manera
    datos["Precios_de_lista"] = datos["Precios_de_lista"].replace(",","",regex=True)
    datos["Precios_en_promoción"]=datos["Precios_en_promoción"].replace(",","",regex=True)
        
    #Convertimos los precios a valores numéricos    
    datos['Precios_de_lista'] = pd.to_numeric(datos['Precios_de_lista'], errors='coerce')
    datos['Precios_en_promoción'] = pd.to_numeric(datos['Precios_en_promoción'], errors='coerce')

    datos.to_excel("df_walmart_limpio.xlsx",index=False)
        
    #Visualizamos los tipos de datos
    print(datos.dtypes)
    return datos


precios_floats_2(df_walmart)

df_walmart=pd.read_excel("df_walmart_limpio.xlsx")
df_walmart

### <span style="color:yellow">MISCOTA</span>

In [ ]:
#Creamos un DF vacio
aux_3=pd.DataFrame()

In [ ]:
#Convertimos el DF a un archivo de excel ".xlsx"
aux_3.to_excel("df_miscota.xlsx",index=False)

In [ ]:
#Función que recibe como parámetro el producto de interés
def Buscador_Precios_Miscota(producto):

    #1.INGRESAMOS A LA PÁGINA WEB
    path ="/Users/Carmen Rosa/Documents/teoria-LP2/web/chromedriver"
    driver=webdriver.Chrome(path)
    url= "https://mx.miscota.com/perros/c_"+producto
    driver.get(url)
    
    #2.ACCEDEMOS A LOS ELEMENTOS QUE CONTIENEN LOS DATOS QUE QUEREMOS DE LA PÁGINA WEB
    productos= driver.find_elements_by_class_name("prod__box")
    
    #3.ACCEDEMOS A LAS URL'S ALMACENADAS EN LA VARIABLE PRODUCTOS
    #Para obtener varios url´s de jalón ocupamos un ciclo for
    #Declaramos una lista vacía
    lista_urls=list()
    #Para cada una de los productos hasta la longitud de la lista:
    for i in range(len(productos)):
        try:
            #En cada una de las entradas se va a buscar la etiqueta "a" y el atributo "href"
            lista_urls.append(productos[i].find_element_by_tag_name("a").get_attribute("href"))
        except:
            #En caso de error se va a agregar a la lista un registro NaN
            lista_urls.append(np.nan)
    
    #4.ACCEDEMOS A LOS NOMBRES ALMACENADAS EN LA VARIABLE PRODUCTOS
    #Ahora hacemos lo mismo pero para obtener los NOMBRES
    lista_nombres=list()
    for i in range(len(productos)):
        try:
            lista_nombres.append(productos[i].find_elements_by_tag_name("a")[0].text)
            #lista_nombres.append(productos[i].find_elements_by_class_name("a")[0].text)
        except:
            lista_nombres.append(np.nan)
            
    #5.ACCEDEMOS A LOS PRECIOS BASE Y PROMO EN LA VARIABLE PRODUCTOS
    lista_precios=list()

    for i in range(len(productos)):

        try:
            lista_precios.append(productos[i].find_elements_by_class_name("prod__box--price")[0].text.split("\n")[0])

        except:
            lista_precios.append(np.nan)
            
    #6.CREAR DATAFRAME A PARTIR DE LOS DATOS OBTENIDOS
    #Creamos el DataFrame
    #Los precios los tomamos como están actualmente en la página. algunos con descuentos y otros sin
    #Así nos dejo entregarlo Fer debido a que había un problema al obetener los precios con y sin descuento por separado
    df_miscota =pd.DataFrame({"Nombre_producto":lista_nombres,
                             "Producto":producto,
                             "Precios_de_lista":lista_precios,
                             "url":lista_urls})

    #A través de la librería "time" se pone la fecha asociada al día en el que se hace la extracción de la información
    df_miscota["Fecha_de_consulta"]= time.strftime("%d/%m/%y")

    df_miscota["Autoservicio"]="Miscota"
    
    #Para reacomodar las columnas del DF
    df_miscota = df_miscota[["Fecha_de_consulta","Autoservicio","Producto","Nombre_producto","Precios_de_lista","url"]]
    
    #7.ESCRIBIR SOBRE EXCEL
    #Convertimos el excel en un DF
    datos_webscraper=pd.read_excel("df_miscota.xlsx")

    #Concatenamos el DF "vacío" con el DF de Miscota con el que estuvimos trabajando
    datos_webscraper= pd.concat([datos_webscraper,df_miscota],axis=0)

    #Finalmente convertimos de nuevo el DF en un archivo excel con el mismo nombre
    datos_webscraper.to_excel("df_miscota.xlsx",index=False)
    
    #8.SALIR DEL DRIVER
    #Para salir del driver (cerrar ventana de chrome con la que estámos trabajando)
    driver.quit()
    #Al final se regresa el DF de Mister Mascotas
    return df_miscota

#Para cada función que esté en la lista se va a llamar a la función
for productos in ["comida","descanso","juguetes-deporte"]:
    Buscador_Precios_Miscota(productos)
    
#Se lee la hoja de cálculo de excel
df_miscota=pd.read_excel("df_miscota.xlsx")
df_miscota

In [ ]:
#PRECIOS (CADENAS) A FLOATS
#Recibe todos los registros del DF 
def precios_floats_3(datos):

    #Eliminamos el signo en los precios
    
    #1ª columna
    for i in range(len(datos["Precios_de_lista"])):
        try:
            datos["Precios_de_lista"].iloc[i]=datos["Precios_de_lista"].iloc[i].strip("$")
        except:
            pass
          
    #Quitamos la separación de comas para miles
    datos["Precios_de_lista"]=datos["Precios_de_lista"].replace(",","",regex=True)
    
   #Convertimos los precios a numéricos    
    datos['Precios_de_lista'] = pd.to_numeric(datos['Precios_de_lista'], errors='coerce')

    datos.to_excel("df_miscota_float.xlsx",index=False)
        
    #Visualizamos los tipos de datos
    print(datos.dtypes)
    return datos

#Verificamos que el DF ahora cuente con precios de tipo float
precios_floats_3(df_miscota)

df_miscota=pd.read_excel("df_miscota_float.xlsx")
df_miscota

In [ ]:
#ELIMINAMOS LAS FILAS CON DATOS VACÍOS
df_miscota = df_miscota[df_miscota['Nombre_producto'].notna()]
print(df_miscota)

In [ ]:
#Reseteamos el índice de la tabla para cada vez que se corra el código antes de que se agregue al Excel
df_miscota=df_miscota.reset_index(drop=True)

In [ ]:
#Creamos un DF vacio
aux_4=pd.DataFrame()

#Convertimos el DF a un archivo de excel ".xlsx"
aux_4.to_excel("df_miscota_limpio.xlsx",index=False)

#Finalmente convertimos de nuevo el DF en un archivo excel con el mismo nombre
df_miscota.to_excel("df_miscota_limpio.xlsx",index=False)

In [ ]:
df_miscota

## <span style="color:maroon">Juntamos todos los DF en el mismo archivo excel</span>

In [ ]:
#Creamos un DF vacio
df_completo=pd.DataFrame()

#Convertimos el DF a un archivo de excel ".xlsx"
df_completo.to_excel("df_completo.xlsx",index=False)

In [ ]:
#Concatenamos los DF "vacío" con el DF vacío que acabamos de crear
df_completo= pd.concat([df_completo,df_mister,df_walmart,df_miscota],axis=0)
df_completo.to_excel("df_completo.xlsx",index=False)

In [ ]:
df_completo